In [13]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [14]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
861,This movie received a great write up in Blockb...,negative
947,The film begins with a little girl (Rita) seei...,negative
730,Who were they kidding with this? There was jus...,negative
824,I normally do not take the time to make commen...,negative
672,USA The Movie is like this: You take a nap on ...,positive


In [15]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [16]:
df = normalize_text(df)
df.head()

,review,sentiment
861,movie received great write blockbuster coming ...,negative
947,film begin little girl rita seeing father kill...,negative
730,kidding this much film hard digest right arjun...,negative
824,normally take time make comment people read mo...,negative
672,usa movie like this take nap long hot sunday a...,positive


In [17]:
df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [18]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [19]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
861,movie received great write blockbuster coming ...,0
947,film begin little girl rita seeing father kill...,0
730,kidding this much film hard digest right arjun...,0
824,normally take time make comment people read mo...,0
672,usa movie like this take nap long hot sunday a...,1


In [20]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [21]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/dhruvchandralohani/Capstone-Project.mlflow')
dagshub.init(repo_owner='dhruvchandralohani', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-11-21 16:48:27,804 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/dhruvchandralohani/Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "dhruvchandralohani/Capstone-Project"

2025-11-21 16:48:27,809 - INFO - Initialized MLflow to track repo "dhruvchandralohani/Capstone-Project"


Repository dhruvchandralohani/Capstone-Project initialized!

2025-11-21 16:48:27,811 - INFO - Repository dhruvchandralohani/Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/b9e1061eafd14ef8a8c49dc791b71923', creation_time=1763721787026, experiment_id='0', last_update_time=1763721787026, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [24]:
import mlflow
import logging
import time
import tempfile, shutil
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # DAGsHub's MLflow endpoint doesn't support the CreateLoggedModel endpoint used
        # by mlflow.sklearn.log_model. Save locally and upload artifacts instead.
        tmp_dir = tempfile.mkdtemp(prefix="sklearn_model_")
        try:
            mlflow.sklearn.save_model(model, tmp_dir)
            mlflow.log_artifacts(tmp_dir, artifact_path="model")
        finally:
            shutil.rmtree(tmp_dir, ignore_errors=True)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")
        mlflow.log_metric("execution_time_seconds", end_time - start_time)

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-11-21 16:48:29,038 - INFO - Starting MLflow run...
2025-11-21 16:48:29,392 - INFO - Logging preprocessing parameters...
2025-11-21 16:48:30,416 - INFO - Initializing Logistic Regression model...
2025-11-21 16:48:30,417 - INFO - Fitting the model...
2025-11-21 16:48:30,436 - INFO - Model training complete.
2025-11-21 16:48:30,437 - INFO - Logging model parameters...
2025-11-21 16:48:30,767 - INFO - Making predictions...
2025-11-21 16:48:30,768 - INFO - Calculating evaluation metrics...
2025-11-21 16:48:30,775 - INFO - Logging evaluation metrics...
2025-11-21 16:48:32,129 - INFO - Saving and logging the model...
2025-11-21 16:48:39,144 - INFO - Model training and logging completed in 9.75 seconds.
2025-11-21 16:48:39,473 - INFO - Accuracy: 0.69
2025-11-21 16:48:39,475 - INFO - Precision: 0.5660377358490566
2025-11-21 16:48:39,475 - INFO - Recall: 0.7894736842105263
2025-11-21 16:48:39,476 - INFO - F1 Score: 0.6593406593406593


🏃 View run ambitious-fox-624 at: https://dagshub.com/dhruvchandralohani/Capstone-Project.mlflow/#/experiments/0/runs/891b51932a77491091a7b1c2ff995e2b
🧪 View experiment at: https://dagshub.com/dhruvchandralohani/Capstone-Project.mlflow/#/experiments/0
